In [1]:
import pandas as pd
import numpy as np
import sys
import os
from datetime import datetime, time
sys.path.append(os.path.abspath(".."))
from core.viz import plot_line, create_subplot_grid, plot_bar, plot_statistical_strip, plot_scatter, plot_distribution
from core.s3 import S3AssetManager

In [2]:
notebook_name = "exp_central_ganadera_tiempos"
s3 = S3AssetManager(notebook_name=notebook_name)

In [3]:
df = pd.read_excel("../raw/RG L 04 Informe Resultado Laboratorio 2025.xlsx",
sheet_name="TRAZABILIDAD PRODUCTO", skiprows=6)
df.columns = df.columns.str.strip().str.lower()

/Users/juandavidrincon/Documents/hawking/.venv/lib/python3.13/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [4]:
rename_map = {
    "#": "id",
    "día muestreo": "dia_muestreo",
    "mes muestreo": "mes_muestreo",
    "año muestreo": "anio",
    "fecha": "fecha",
    "hora \ntoma de muestra": "hora_toma_muestra",
    "municipio": "municipio",
    "granja": "granja",
    "productor/cliente planta": "productor_cliente_planta",
    "lugar\ntoma de muestra": "lugar_toma_muestra",
    "día\ning lab": "dia_ingreso_lab",
    "hora \ning lab": "hora_ingreso_lab",
    "tipo de muestra\nproducto": "tipo_muestra_producto",
    "id muestra": "id_muestra",
    "#cava #muestra": "cava_muestra",
    "t°c/ frío": "temperatura_c_frio",
    "responsable muestreo": "responsable_muestreo",
    "procesado por": "procesado_por",
    "día \nsiembra / incubadora": "dia_siembra_incubadora",
    "hora incubación": "hora_incubacion",

    "análisis realizado e. coli\nincubación a 36°c por 24 horas": "analisis_ecoli_36c_24h",
    "conteo placa ec": "conteo_placa_ec",
    "resultado ec ufc/cm2": "resultado_ec_ufc_cm2",
    "m ec": "m_ec",
    "m ec4": "m_ec4",
    "interpretación ec": "interpretacion_ec",

    "análisis realizado coliformes totales\n incubación a 36°c por 24 horas": "analisis_coliformes_totales_36c_24h",
    "conteo placa ct": "conteo_placa_ct",
    "resultado ct ufc/cm2": "resultado_ct_ufc_cm2",
    "m ct": "m_ct",
    "m ct5": "m_ct5",
    "interpretación ct": "interpretacion_ct",

    "análisis realizado salmonella spp\nincubación a 41,5°c por 24 horas": "analisis_salmonella_spp_41_5c_24h",
    "resultado salmonella": "resultado_salmonella",
    "interpretación salmonella": "interpretacion_salmonella",
    "confirmación laboratorio externo salmonella": "confirmacion_lab_externo_salmonella",

    "análisis realizado e. coli o157:h7\nincubación a 41,5°c por 24 horas": "analisis_ecoli_o157_h7_41_5c_24h",
    "resultado ec o157:h7": "resultado_ecoli_o157_h7",
    "interpretación ec o157:h7": "interpretacion_ecoli_o157_h7",
    "confirmación laboratorio externo ec o157:h7": "confirmacion_lab_externo_ecoli_o157_h7",

    "obs": "observaciones",
    "dato erróneo que se corrigió (escriba el dato malo aquí)": "dato_erroneo_corregido",
    "responsable del error": "responsable_error",
    "fecha en la que se hace la corrección": "fecha_correccion",
    "responsable corrección": "responsable_correccion",
    "revisado por": "revisado_por",
    "fecha revisión": "fecha_revision",
}

df = df.rename(columns=rename_map)


In [5]:
df.columns

Index(['id', 'dia_muestreo', 'mes_muestreo', 'anio', 'fecha',
       'hora_toma_muestra', 'municipio', 'granja', 'productor_cliente_planta',
       'lugar_toma_muestra', 'dia_ingreso_lab', 'hora_ingreso_lab',
       'tipo_muestra_producto', 'id_muestra', 'cava_muestra',
       'temperatura_c_frio', 'responsable_muestreo', 'procesado_por',
       'dia_siembra_incubadora', 'hora_incubacion', 'analisis_ecoli_36c_24h',
       'conteo_placa_ec', 'resultado_ec_ufc_cm2', 'm_ec', 'm_ec4',
       'interpretacion_ec', 'analisis_coliformes_totales_36c_24h',
       'conteo_placa_ct', 'resultado_ct_ufc_cm2', 'm_ct', 'm_ct5',
       'interpretacion_ct', 'analisis_salmonella_spp_41_5c_24h',
       'resultado_salmonella', 'interpretacion_salmonella',
       'confirmacion_lab_externo_salmonella',
       'analisis_ecoli_o157_h7_41_5c_24h', 'resultado_ecoli_o157_h7',
       'interpretacion_ecoli_o157_h7',
       'confirmacion_lab_externo_ecoli_o157_h7', 'observaciones',
       'dato_erroneo_corregido', '

In [6]:
def assemble_dt(row, d_col, t_col):
    try:
        # Extraer año, mes y día (asegurando enteros)
        y = int(row['anio'])
        m = int(row['mes_muestreo'])
        d = int(row[d_col])
        
        # Manejar la hora
        t_val = row[t_col]
        
        # Si ya es un objeto time de Python
        if isinstance(t_val, time):
            h, mi, s = t_val.hour, t_val.minute, t_val.second
        else:
            # Si es un string o float, intentar partirlo
            parts = str(t_val).split(':')
            h = int(float(parts[0]))
            mi = int(float(parts[1])) if len(parts) > 1 else 0
            s = int(float(parts[2])) if len(parts) > 2 else 0
            
        return datetime(y, m, d, h, mi, s)
    except:
        return pd.NaT

In [7]:

df['sample_date'] = df.apply(lambda r: assemble_dt(r, 'dia_muestreo', 'hora_toma_muestra'), axis=1)
df['fecha_ingreso'] = df.apply(lambda r: assemble_dt(r, 'dia_ingreso_lab', 'hora_ingreso_lab'), axis=1)
df['fecha_incubacion'] = df.apply(lambda r: assemble_dt(r, 'dia_siembra_incubadora', 'hora_incubacion'), axis=1)

df['horas_toma_a_ingreso'] = (df['fecha_ingreso'] - df['sample_date']).dt.total_seconds() / 3600
df['horas_ingreso_a_incubacion'] = (df['fecha_incubacion'] - df['fecha_ingreso']).dt.total_seconds() / 3600
df['horas_toma_a_incubacion'] = (df['fecha_incubacion'] - df['sample_date']).dt.total_seconds() / 3600
df['year_month'] = df['fecha'].dt.to_period('M')

In [8]:
cond = (df['horas_toma_a_incubacion']<0) | (df['horas_ingreso_a_incubacion']<0) | (df['horas_toma_a_ingreso']<0)
df_good = df[~cond]
df_bad = df[cond]

In [9]:
df

,id,dia_muestreo,mes_muestreo,anio,fecha,hora_toma_muestra,municipio,granja,productor_cliente_planta,lugar_toma_muestra,...,responsable_correccion,revisado_por,fecha_revision,sample_date,fecha_ingreso,fecha_incubacion,horas_toma_a_ingreso,horas_ingreso_a_incubacion,horas_toma_a_incubacion,year_month
0,P00001,2.0,1.0,2025,2025-01-02,22:20:00,VALPARAISO,LA MACHONTA,ALIMENTOS CARNICOS S.A.S,LP,...,NaN,NaN,NaN,2025-01-02 22:20:00,2025-01-02 22:30:00,2025-01-03 08:30:00,0.166667,10.000000,10.166667,2025-01
1,P00002,2.0,1.0,2025,2025-01-02,23:22:00,PUERTO BERRÍO,PUENTE ESCOCIA #2,ENSAYO ENSAYO,LB,...,NaN,NaN,NaN,2025-01-02 23:22:00,2025-01-02 00:00:00,2025-01-03 08:10:00,-23.366667,32.166667,8.800000,2025-01
2,P00003,2.0,1.0,2025,2025-01-02,23:41:00,NaN,NaN,NaN,FB,...,NaN,NaN,NaN,2025-01-02 23:41:00,2025-01-02 23:54:00,2025-01-07 10:36:00,0.216667,106.700000,106.916667,2025-01
3,P00004,2.0,1.0,2025,2025-01-02,23:43:00,NaN,NaN,NaN,FB,...,NaN,NaN,NaN,2025-01-02 23:43:00,2025-01-02 00:00:00,2025-01-07 10:30:00,-23.716667,130.500000,106.783333,2025-01
4,P00005,3.0,1.0,2025,2025-01-03,00:52:00,PUEBLO NUEVO,LA EXTENSION,CARLOS PELAEZ,LB,...,NaN,NaN,NaN,2025-01-03 00:52:00,2025-01-03 01:05:00,2025-01-03 08:15:00,0.216667,7.166667,7.383333,2025-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8062,P08063,NaN,NaN,2025,2024-11-30,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,2024-11
8063,P08064,NaN,NaN,2025,2024-11-30,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,2024-11
8064,P08065,NaN,NaN,2025,2024-11-30,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,2024-11
8065,P08066,NaN,NaN,2025,2024-11-30,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,2024-11


In [10]:
cls = [
'id', 
'fecha', 
#'dia_muestreo',
#'mes_muestreo', 
#'anio',
#'hora_toma_muestra', 
#'dia_ingreso_lab', 
#'hora_ingreso_lab',
#'sample_date',
'horas_toma_a_ingreso', 
'horas_ingreso_a_incubacion',
'horas_toma_a_incubacion',
#'fecha_ingreso',
#'fecha_incubacion',
'tipo_muestra_producto', 
#'cava_muestra',
#'dia_siembra_incubadora',
#'hora_incubacion', 
'analisis_ecoli_36c_24h',
'resultado_ec_ufc_cm2',
'analisis_coliformes_totales_36c_24h',
'resultado_ct_ufc_cm2',
'analisis_salmonella_spp_41_5c_24h',
'resultado_salmonella',
'analisis_ecoli_o157_h7_41_5c_24h',
'resultado_ecoli_o157_h7',
]
df_good[cls]


,id,fecha,horas_toma_a_ingreso,horas_ingreso_a_incubacion,horas_toma_a_incubacion,tipo_muestra_producto,analisis_ecoli_36c_24h,resultado_ec_ufc_cm2,analisis_coliformes_totales_36c_24h,resultado_ct_ufc_cm2,analisis_salmonella_spp_41_5c_24h,resultado_salmonella,analisis_ecoli_o157_h7_41_5c_24h,resultado_ecoli_o157_h7
0,P00001,2025-01-02,0.166667,10.000000,10.166667,Canal Porcina,Petrifilm EC: E. coli gen,1.50,Petrifilm EC: Col Tot,2.0,No realizado,NaN,No realizado,NaN
2,P00003,2025-01-02,0.216667,106.700000,106.916667,Canal Bovina,No realizado,NaN,No realizado,NaN,No realizado,NaN,mini VIDAS E. coli O157:H7,0.0
4,P00005,2025-01-03,0.216667,7.166667,7.383333,Canal Bovina,Petrifilm EC: E. coli gen,0.25,Petrifilm EC: Col Tot,0.0,No realizado,NaN,No realizado,NaN
5,P00006,2025-01-03,0.250000,7.416667,7.666667,Canal Porcina,Petrifilm EC: E. coli gen,0.42,Petrifilm EC: Col Tot,1.0,No realizado,NaN,No realizado,NaN
6,P00007,2025-01-03,0.166667,6.166667,6.333333,Canal Bovina,Petrifilm EC: E. coli gen,0.08,Petrifilm EC: Col Tot,0.0,No realizado,NaN,No realizado,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8062,P08063,2024-11-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8063,P08064,2024-11-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8064,P08065,2024-11-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8065,P08066,2024-11-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df_good["resultado_ec_ufc_cm2"].describe()

count    3333.000000
mean       11.246406
std       123.677757
min         0.000000
25%         0.000000
50%         0.000000
75%         0.170000
max      3000.000000
Name: resultado_ec_ufc_cm2, dtype: float64

In [12]:
df_good["resultado_ct_ufc_cm2"].describe()

count    3328.000000
mean       23.198618
std       167.239614
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max      3000.000000
Name: resultado_ct_ufc_cm2, dtype: float64

In [13]:
df_good[df_good['analisis_ecoli_36c_24h']=='Petrifilm EC: E. coli gen']["resultado_ec_ufc_cm2"].describe()

count    3333.000000
mean       11.246406
std       123.677757
min         0.000000
25%         0.000000
50%         0.000000
75%         0.170000
max      3000.000000
Name: resultado_ec_ufc_cm2, dtype: float64

In [14]:
cls_date = [
'id', 
'fecha', 
'dia_muestreo',
'mes_muestreo', 
'anio',
'hora_toma_muestra', 
'dia_ingreso_lab', 
'hora_ingreso_lab',
'fecha_ingreso',
'fecha_incubacion',
'dia_siembra_incubadora',
]
df_bad[cls_date]

,id,fecha,dia_muestreo,mes_muestreo,anio,hora_toma_muestra,dia_ingreso_lab,hora_ingreso_lab,fecha_ingreso,fecha_incubacion,dia_siembra_incubadora
1,P00002,2025-01-02,2.0,1.0,2025,23:22:00,2.0,00:00:00,2025-01-02 00:00:00,2025-01-03 08:10:00,3.0
3,P00004,2025-01-02,2.0,1.0,2025,23:43:00,2.0,00:00:00,2025-01-02 00:00:00,2025-01-07 10:30:00,7.0
26,P00027,2025-01-06,6.0,1.0,2025,23:22:00,6.0,00:05:00,2025-01-06 00:05:00,2025-01-07 09:46:00,7.0
36,P00037,2025-01-07,7.0,1.0,2025,02:28:00,8.0,02:38:00,2025-01-08 02:38:00,2025-01-07 10:48:00,7.0
37,P00038,2025-01-07,7.0,1.0,2025,02:32:00,9.0,02:45:00,2025-01-09 02:45:00,2025-01-07 02:45:00,7.0
...,...,...,...,...,...,...,...,...,...,...,...
5084,P05085,2025-11-29,29.0,11.0,2025,11:20:00,29.0,11:35:00,2025-11-29 11:35:00,2025-11-01 13:56:00,1.0
5085,P05086,2025-11-29,29.0,11.0,2025,22:46:00,29.0,22:58:00,2025-11-29 22:58:00,2025-11-01 14:00:00,1.0
5086,P05087,2025-11-29,29.0,11.0,2025,22:49:00,29.0,22:58:00,2025-11-29 22:58:00,2025-11-01 14:02:00,1.0
5087,P05088,2025-11-30,30.0,11.0,2025,22:50:00,29.0,23:10:00,2025-11-29 23:10:00,2025-11-01 14:04:00,1.0


In [15]:
df_good["analisis_ecoli_36c_24h"].value_counts(dropna=False)

analisis_ecoli_36c_24h
Petrifilm EC: E. coli gen    5858
No realizado                 1380
NaN                           450
Name: count, dtype: int64

In [16]:
df_good["horas_ingreso_a_incubacion"].describe()

count    4717.000000
mean       18.017303
std        22.228710
min         0.000000
25%         6.900000
50%        10.350000
75%        14.216667
max       129.366667
Name: horas_ingreso_a_incubacion, dtype: float64

In [17]:
df["year_month"].value_counts()

year_month
2024-11    2922
2025-10     514
2025-05     498
2025-07     485
2025-06     482
2025-11     481
2025-09     477
2025-08     476
2025-03     452
2025-01     443
2025-04     431
2025-02     406
Freq: M, Name: count, dtype: int64

In [18]:
fig1 = plot_statistical_strip(
    df=df_good,
    x_col='lugar_toma_muestra',       # Category: Product Type
    y_col='horas_ingreso_a_incubacion',        # Value: Coliforms Count
    title="Carga de Coliformes Totales por Tipo de Producto",
    y_title="UFC/cm2",
    x_title="Producto",
    point_opacity=0.5,
    show_mean_ci=True,                   # Show the mean and confidence interval
    show_boxplot=True                    # Show the distribution box
)

fig1.show()

In [44]:
fig1 = plot_statistical_strip(
    df=df_good,
    x_col='tipo_muestra_producto',       # Category: Product Type
    y_col='horas_ingreso_a_incubacion',        # Value: Coliforms Count
    title="Carga de Coliformes Totales por Tipo de Producto",
    y_title="UFC/cm2",
    x_title="Producto",
    point_opacity=0.5,
    show_mean_ci=True,                   # Show the mean and confidence interval
    show_boxplot=True                    # Show the distribution box
)

fig1.show()

In [19]:
fig1 = plot_distribution(
    df=df_good,
    group_col="tipo_muestra_producto",
    value_col="horas_ingreso_a_incubacion",
    title="Histograma de Coliformes Totales",
    marginal="box",  # Muestra un boxplot arriba para ver outliers
    nbins=100         # Forzar 30 barras
)
fig1.show()

In [20]:
fig1 = plot_distribution(
    df=df_good,
    group_col="year_month",
    value_col="horas_ingreso_a_incubacion",
    title="Histograma de Coliformes Totales",
    marginal="box",  # Muestra un boxplot arriba para ver outliers
    nbins=100         # Forzar 30 barras
)
fig1.show()

In [21]:
# Asegúrate primero que la columna sea string para que se ordene bien
df_good['year_month'] = df_good['year_month'].astype(str)

fig1 = plot_statistical_strip(
    df=df_good,                        # <--- CORRECCIÓN: Pasa el DF limpio
    x_col='year_month',                # Eje X: Meses
    y_col='horas_ingreso_a_incubacion',# Eje Y: Horas
    # Títulos actualizados para que tengan sentido con tus datos
    title="Tiempo de Ingreso a Incubación por Mes",
    y_title="Horas",
    x_title="Mes",
    
    point_opacity=0.5,
    show_mean_ci=True,
    show_boxplot=True,
    show_counts=True                   # Útil para ver cuántas muestras hubo por mes
)

fig1.show()

/var/folders/1g/77kw2x4j5678s_87_sqc1fpc0000gp/T/ipykernel_24520/3964355493.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [28]:
analisis_ecoli_36c_24h = df_good[df_good["analisis_ecoli_36c_24h"] == 'Petrifilm EC: E. coli gen'].copy()

cols_micro = [
    'resultado_ec_ufc_cm2', 
    'resultado_ct_ufc_cm2', 
    'resultado_ecoli_o157_h7'
]

for col in cols_micro:
    # Convertimos a numérico, los errores se vuelven NaN. 
    # IMPORTANTE: Ya NO usamos .fillna(0)
    analisis_ecoli_36c_24h[col] = pd.to_numeric(analisis_ecoli_36c_24h[col], errors='coerce')
    
    # Aplicamos logaritmo:
    # Si el valor es 10 -> log10(11) ≈ 1.04
    # Si el valor es 0 -> log10(1) = 0  
    # Si el valor es NaN -> NaN + 1 = NaN -> log10(NaN) = NaN (Se conserva el nulo)
    analisis_ecoli_36c_24h[f'{col}_log'] = np.log10(analisis_ecoli_36c_24h[col] + 1)

analisis_ecoli_36c_24h


,id,dia_muestreo,mes_muestreo,anio,fecha,hora_toma_muestra,municipio,granja,productor_cliente_planta,lugar_toma_muestra,...,sample_date,fecha_ingreso,fecha_incubacion,horas_toma_a_ingreso,horas_ingreso_a_incubacion,horas_toma_a_incubacion,year_month,resultado_ec_ufc_cm2_log,resultado_ct_ufc_cm2_log,resultado_ecoli_o157_h7_log
0,P00001,2.0,1.0,2025,2025-01-02,22:20:00,VALPARAISO,LA MACHONTA,ALIMENTOS CARNICOS S.A.S,LP,...,2025-01-02 22:20:00,2025-01-02 22:30:00,2025-01-03 08:30:00,0.166667,10.000000,10.166667,2025-01,0.397940,0.477121,NaN
4,P00005,3.0,1.0,2025,2025-01-03,00:52:00,PUEBLO NUEVO,LA EXTENSION,CARLOS PELAEZ,LB,...,2025-01-03 00:52:00,2025-01-03 01:05:00,2025-01-03 08:15:00,0.216667,7.166667,7.383333,2025-01,0.096910,0.000000,NaN
5,P00006,3.0,1.0,2025,2025-01-03,00:55:00,SANTA ROSA DE OSOS,EL PORVENIR - VELSAGRO SAS,INVERSIONES BARILOCHE JYM SAS,LP,...,2025-01-03 00:55:00,2025-01-03 01:10:00,2025-01-03 08:35:00,0.250000,7.416667,7.666667,2025-01,0.152288,0.301030,NaN
6,P00007,3.0,1.0,2025,2025-01-03,01:50:00,CAÑASGORDAS,RUBICON,RAMIRO MEDINA,LB,...,2025-01-03 01:50:00,2025-01-03 02:00:00,2025-01-03 08:10:00,0.166667,6.166667,6.333333,2025-01,0.033424,0.000000,NaN
7,P00008,3.0,1.0,2025,2025-01-03,01:57:00,Caldas,LA FLORA,AGROSIGO SAS,CAVA VRP,...,2025-01-03 01:57:00,2025-01-03 02:10:00,2025-01-03 02:30:00,0.216667,0.333333,0.550000,2025-01,0.397940,0.602060,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7612,P07613,NaN,NaN,2025,2024-11-30,NaN,NaN,NaN,NaN,NaN,...,NaT,NaT,NaT,NaN,NaN,NaN,2024-11,NaN,NaN,NaN
7613,P07614,NaN,NaN,2025,2024-11-30,NaN,NaN,NaN,NaN,NaN,...,NaT,NaT,NaT,NaN,NaN,NaN,2024-11,NaN,NaN,NaN
7614,P07615,NaN,NaN,2025,2024-11-30,NaN,NaN,NaN,NaN,NaN,...,NaT,NaT,NaT,NaN,NaN,NaN,2024-11,NaN,NaN,NaN
7615,P07616,NaN,NaN,2025,2024-11-30,NaN,NaN,NaN,NaN,NaN,...,NaT,NaT,NaT,NaN,NaN,NaN,2024-11,NaN,NaN,NaN


In [36]:
#[analisis_ecoli_36c_24h["resultado_ec_ufc_cm2"]>0]
plot_scatter(
    df=analisis_ecoli_36c_24h,
     x_col="horas_ingreso_a_incubacion",
      y_col="resultado_ec_ufc_cm2_log",
       group_col="tipo_muestra_producto", 
       size_col="", 
       order_groups=None, 
       hover_data_cols=None, 
       title="", 
       x_title=None, 
       y_title=None, 
       text_format=".1f", 
       marker_size=10, 
       marker_opacity=0.8, 
       marker_colors=None, 
       height=600, 
       width=1000, 
       font_family="Inter, Arial, sans-serif", 
       output_path=None
)

In [104]:
plot_scatter(
    df=analisis_ecoli_36c_24h[analisis_ecoli_36c_24h["resultado_ec_ufc_cm2"]>0],
     x_col="horas_toma_a_ingreso",
      y_col="horas_toma_a_incubacion",
       group_col="tipo_muestra_producto", 
       size_col="", 
       order_groups=None, 
       hover_data_cols=None, 
       title="", 
       x_title=None, 
       y_title=None, 
       text_format=".1f", 
       marker_size=10, 
       marker_opacity=0.8, 
       marker_colors=None, 
       height=600, 
       width=1000, 
       font_family="Inter, Arial, sans-serif", 
       output_path=None
)

In [107]:
salmo = df_good[df_good["resultado_salmonella"].notnull()][cls]
salmo

,id,horas_toma_a_ingreso,horas_ingreso_a_incubacion,horas_toma_a_incubacion,tipo_muestra_producto,analisis_ecoli_36c_24h,resultado_ec_ufc_cm2,analisis_coliformes_totales_36c_24h,resultado_ct_ufc_cm2,analisis_salmonella_spp_41_5c_24h,resultado_salmonella,analisis_ecoli_o157_h7_41_5c_24h,resultado_ecoli_o157_h7
8,P00009,0.233333,104.233333,104.466667,Canal Porcina,No realizado,NaN,No realizado,NaN,mini VIDAS Salmonella spp,0.0,No realizado,NaN
11,P00012,0.200000,96.000000,96.200000,Canal Bovina >7 °c,No realizado,NaN,No realizado,NaN,mini VIDAS Salmonella spp,0.0,No realizado,NaN
13,P00014,0.250000,85.366667,85.616667,Canal Porcina,No realizado,NaN,No realizado,NaN,mini VIDAS Salmonella spp,0.0,No realizado,NaN
16,P00017,0.183333,82.700000,82.883333,Canal Bovina,No realizado,NaN,No realizado,NaN,mini VIDAS Salmonella spp,0.0,No realizado,NaN
23,P00024,0.250000,12.650000,12.900000,Carne Cabeza Bovina,Petrifilm EC: E. coli gen,260.0,Petrifilm EC: Col Tot,390.0,mini VIDAS Salmonella spp,0.0,No realizado,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5035,P05036,0.250000,9.950000,10.200000,VB Bovina,Petrifilm EC: E. coli gen,0.0,Petrifilm EC: Col Tot,0.0,Mini VIDAS Salmonella spp,0.0,No realizado,NaN
5038,P05039,0.150000,7.966667,8.116667,Canal Bovina >7 °c,No realizado,NaN,No realizado,NaN,Mini VIDAS Salmonella spp,0.0,No realizado,NaN
5043,P05044,0.500000,16.033333,16.533333,Canal Bovina >7 °c,No realizado,NaN,No realizado,NaN,Mini VIDAS Salmonella spp,0.0,No realizado,NaN
5045,P05046,10.433333,16.100000,26.533333,Canal Bovina,No realizado,NaN,No realizado,NaN,Mini VIDAS Salmonella spp,0.0,No realizado,NaN


In [112]:
plot_scatter(
    df=salmo,#[salmo["resultado_salmonella"]>0],
     x_col="horas_toma_a_ingreso",
      y_col="resultado_salmonella",
       group_col="tipo_muestra_producto", 
       size_col="", 
       order_groups=None, 
       hover_data_cols=None, 
       title="", 
       x_title=None, 
       y_title=None, 
       text_format=".1f", 
       marker_size=10, 
       marker_opacity=0.8, 
       marker_colors=None, 
       height=600, 
       width=1000, 
       font_family="Inter, Arial, sans-serif", 
       output_path=None
)

In [113]:
df_good[cls]

,id,horas_toma_a_ingreso,horas_ingreso_a_incubacion,horas_toma_a_incubacion,tipo_muestra_producto,analisis_ecoli_36c_24h,resultado_ec_ufc_cm2,analisis_coliformes_totales_36c_24h,resultado_ct_ufc_cm2,analisis_salmonella_spp_41_5c_24h,resultado_salmonella,analisis_ecoli_o157_h7_41_5c_24h,resultado_ecoli_o157_h7
0,P00001,0.166667,10.000000,10.166667,Canal Porcina,Petrifilm EC: E. coli gen,1.50,Petrifilm EC: Col Tot,2.0,No realizado,NaN,No realizado,NaN
2,P00003,0.216667,106.700000,106.916667,Canal Bovina,No realizado,NaN,No realizado,NaN,No realizado,NaN,mini VIDAS E. coli O157:H7,0.0
4,P00005,0.216667,7.166667,7.383333,Canal Bovina,Petrifilm EC: E. coli gen,0.25,Petrifilm EC: Col Tot,0.0,No realizado,NaN,No realizado,NaN
5,P00006,0.250000,7.416667,7.666667,Canal Porcina,Petrifilm EC: E. coli gen,0.42,Petrifilm EC: Col Tot,1.0,No realizado,NaN,No realizado,NaN
6,P00007,0.166667,6.166667,6.333333,Canal Bovina,Petrifilm EC: E. coli gen,0.08,Petrifilm EC: Col Tot,0.0,No realizado,NaN,No realizado,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8062,P08063,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8063,P08064,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8064,P08065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8065,P08066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
plot_scatter(
    df=df_good[df_good["resultado_ct_ufc_cm2"]>100],
     x_col="horas_ingreso_a_incubacion",
      y_col="resultado_ct_ufc_cm2",
       group_col="tipo_muestra_producto", 
       size_col="", 
       order_groups=None, 
       hover_data_cols=None, 
       title="", 
       x_title=None, 
       y_title=None, 
       text_format=".1f", 
       marker_size=10, 
       marker_opacity=0.8, 
       marker_colors=None, 
       height=600, 
       width=1000, 
       font_family="Inter, Arial, sans-serif", 
       output_path=None
)